<a href="https://colab.research.google.com/github/dharma610/Project/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
!pip install mpld3
import mpld3
mpld3.enable_notebook()

     |████████████████████████████████| 1.0MB 2.7MB/s 
  Created wheel for mpld3: filename=mpld3-0.5.1-cp36-none-any.whl size=364065 sha256=400e5b488fb4fb51bbcb1231d37853be31257c68207d59115b7015cf8ac3a162
  Stored in directory: /root/.cache/pip/wheels/38/68/06/d119af6c3f9a2d1e123c1f72d276576b457131b3a7bf94e402
Successfully built mpld3


In [6]:
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [7]:
def plotsir(t, S, I, R):
  f, ax = plt.subplots(1,1,figsize=(10,4))
  ax.plot(t, S, 'b', alpha=0.7, linewidth=2, label='Susceptible')
  ax.plot(t, I, 'y', alpha=0.7, linewidth=2, label='Infected')
  ax.plot(t, R, 'g', alpha=0.7, linewidth=2, label='Recovered')

  ax.set_xlabel('Time (days)')

  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True, which='major', c='w', lw=2, ls='-')
  legend = ax.legend()
  legend.get_frame().set_alpha(0.5)
  for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  plt.show();

In [8]:
import random
gamma = []
for i in range(1000):
    gamma.append(random.uniform(0, 1))
    


In [9]:
beta = []
for i in range(1000):
  beta.append(2*gamma[i])  #Parameters can be seen as list beta and gamma

In [10]:
P = []
Q = []
M  = []
for i in range(1000): #Plotted for 2 sets
  N = 1000
  S0, I0, R0 = 999, 1, 0  # initial conditions: one infected, rest susceptible
  t = np.linspace(0, 99, 100) # Grid of time points (in days)
  y0 = S0, I0, R0 # Initial conditions vector
  # Integrate the SIR equations over the time grid, t.
  ret = odeint(deriv, y0, t, args=(N, beta[i], gamma[i]))
  S, I, R = ret.T
  P.append(S)
  Q.append(I)
  M.append(R)

  

In [11]:
import pandas as pd

In [12]:
S_value = pd.DataFrame(P)
S_value.to_csv("Susceptible.csv")

In [13]:
I_value = pd.DataFrame(Q)
I_value.to_csv("Infected.csv")

In [14]:
R_value = pd.DataFrame(M)
R_value.to_csv("Recovered.csv")

In [15]:
S_value

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,999.0,996.053784,988.846594,971.601965,932.427481,852.931979,722.356823,565.050179,428.524936,335.391860,279.210471,246.766272,228.186137,217.516335,211.360988,207.796645,205.727344,204.524036,203.823608,203.415655,203.177966,203.039450,202.958719,202.911663,202.884234,202.868246,202.858926,202.853493,202.850326,202.848480,202.847404,202.846776,202.846411,202.846198,202.846073,202.846001,202.845959,202.845934,202.845920,202.845911,...,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900
1,999.0,997.330753,994.282358,988.748017,978.806812,961.285377,931.407279,883.192984,811.811956,718.234325,612.604287,510.551039,424.380802,358.364629,310.698525,277.321991,254.256168,238.377720,227.443667,219.900494,214.685535,211.073285,208.567315,206.826760,205.616775,204.775092,204.189338,203.781561,203.497620,203.299875,203.162145,203.066207,202.999377,202.952822,202.920389,202.897795,202.882054,202.871087,202.863448,202.858125,...,202.845906,202.845904,202.845903,202.845902,202.845901,202.845901,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900
2,999.0,998.700125,998.355467,997.959394,997.504312,996.981530,996.381110,995.691694,994.900322,993.992216,992.950557,991.756223,990.387525,988.819908,987.025642,984.973500,982.628437,979.951271,976.898389,973.421496,969.467444,964.978145,959.890653,954.137420,947.646813,940.343934,932.151819,922.993056,912.791887,901.476810,888.983663,875.259148,860.264655,843.980221,826.408340,807.577327,787.543866,766.394393,744.245001,721.239675,...,318.918703,309.480754,300.790488,292.795727,285.446013,278.693077,272.491142,266.797096,261.570569,256.773935,252.372257,248.333198,244.626900,241.225850,238.104737,235.240306,232.611210,230.197871,227.982345,225.948186,224.080333,222.364991,220.789524,219.342365,218.012916,216.791474,215.669148,214.637795,213.689952,212.818779,212.018009,211.281891,210.605158,209.982973,209.410903,208.884878,208.401165,207.956336,207.547245,207.171003
3,999.0,996.191321,989.506310,973.906445,939.115574,868.719269,750.180258,599.185168,458.773390,357.262868,293.615919,255.940767,233.970884,221.153208,213.643863,209.226981,206.621495,205.081589,204.170368,203.630764,203.311080,203.121634,203.009351,202.942794,202.903340,202.879952,202.866087,202.857867,202.852994,202.850106,202.848393,202.847378,202.846776,202.846419,202.846208,202.846082,202.846008,202.845964,202.845938,202.845922,...,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900,202.845900
4,999.0,996.484604,990.853755,978.439265,951.960954,899.243911,807.125667,676.753691,535.817992,418.081019,336.020725,283.985349,252.176962,232.905716,221.215963,214.099024,209.751117,207.087970,205.453847,204.449968,203.832802,203.453201,20

In [16]:
df1 = pd.concat([S_value, S_value[[2,6,9,27,37,43,56,79,90,95]].T.stack().reset_index(name='new')['new']], axis=1)

In [17]:
Df1 = pd.concat([S_value, S_value[[3,7,10,28,38,44,57,80,91,96]].T.stack().reset_index(name='new')['new']], axis=1)

In [18]:
df1["new"].head(1001)

0       988.846594
1       994.282358
2       998.355467
3       989.506310
4       990.853755
           ...    
996     997.259242
997     998.927341
998     991.968061
999     997.927502
1000    722.356823
Name: new, Length: 1001, dtype: float64

In [19]:
df2 = pd.concat([I_value, I_value[[2,6,9,27,37,43,56,79,90,95]].T.stack().reset_index(name='new')['new']], axis=1)

In [20]:
Df2 = pd.concat([I_value, I_value[[3,7,10,28,38,44,57,80,91,96]].T.stack().reset_index(name='new')['new']], axis=1)

In [21]:
df2["new"]

0       6.045620e+00
1       3.350867e+00
2       1.321840e+00
3       5.719372e+00
4       5.052331e+00
            ...     
9995    1.477279e-07
9996    4.801898e-04
9997    5.378921e+00
9998   -5.642916e-11
9999    1.263303e-01
Name: new, Length: 10000, dtype: float64

In [22]:
df3 = pd.concat([R_value, R_value[[2,6,9,27,37,43,56,79,90,95]].T.stack().reset_index(name='new')['new']], axis=1)

In [23]:
Df3 = pd.concat([R_value, R_value[[3,7,10,28,38,44,57,80,91,96]].T.stack().reset_index(name='new')['new']], axis=1)

In [24]:
df3["new"]

0         5.107785
1         2.366775
2         0.322693
3         4.774319
4         4.093914
           ...    
9995    797.154100
9996    797.153292
9997      4.426814
9998    797.154100
9999    796.941503
Name: new, Length: 10000, dtype: float64

In [25]:
data = pd.DataFrame(df1["new"])

In [26]:
data["new"]

0       988.846594
1       994.282358
2       998.355467
3       989.506310
4       990.853755
           ...    
9995    202.845900
9996    202.846227
9997    990.194264
9998    202.845900
9999    202.932167
Name: new, Length: 10000, dtype: float64

In [27]:
data["suspectible"] = data["new"]

In [28]:
data["infected"] = df2["new"]

In [29]:
data["recovered"] = df3["new"]

In [30]:
data.drop("new",axis = 1,inplace = True)

In [31]:
data

,suspectible,infected,recovered
0,988.846594,6.045620e+00,5.107785
1,994.282358,3.350867e+00,2.366775
2,998.355467,1.321840e+00,0.322693
3,989.506310,5.719372e+00,4.774319
4,990.853755,5.052331e+00,4.093914
...,...,...,...
9995,202.845900,1.477279e-07,797.154100
9996,202.846227,4.801898e-04,797.153292
9997,990.194264,5.378921e+00,4.426814
9998,202.845900,-5.642916e-11,797.154100


In [32]:
df4 = pd.DataFrame(beta)

In [33]:
df5 = pd.DataFrame(gamma)

In [34]:
df4[0]

0      1.816185
1      1.216265
2      0.279756
3      1.759483
4      1.632993
         ...   
995    0.914129
996    0.627503
997    0.035723
998    1.515201
999    0.430095
Name: 0, Length: 1000, dtype: float64

In [35]:
df4[1] = df4[0]
df4[2] = df4[0]
df4[3] = df4[0]
df4[4] = df4[0]
df4[5] = df4[0]
df4[6] = df4[0]
df4[7] = df4[0]
df4[8] = df4[0]
df4[9] = df4[0]


In [36]:
df5[1] = df5[0]
df5[2] = df5[0]
df5[3] = df5[0]
df5[4] = df5[0]
df5[5] = df5[0]
df5[6] = df5[0]
df5[7] = df5[0]
df5[8] = df5[0]
df5[9] = df5[0]


In [37]:
df4

,0,1,2,3,4,5,6,7,8,9
0,1.816185,1.816185,1.816185,1.816185,1.816185,1.816185,1.816185,1.816185,1.816185,1.816185
1,1.216265,1.216265,1.216265,1.216265,1.216265,1.216265,1.216265,1.216265,1.216265,1.216265
2,0.279756,0.279756,0.279756,0.279756,0.279756,0.279756,0.279756,0.279756,0.279756,0.279756
3,1.759483,1.759483,1.759483,1.759483,1.759483,1.759483,1.759483,1.759483,1.759483,1.759483
4,1.632993,1.632993,1.632993,1.632993,1.632993,1.632993,1.632993,1.632993,1.632993,1.632993
...,...,...,...,...,...,...,...,...,...,...
995,0.914129,0.914129,0.914129,0.914129,0.914129,0.914129,0.914129,0.914129,0.914129,0.914129
996,0.627503,0.627503,0.627503,0.627503,0.627503,0.627503,0.627503,0.627503,0.627503,0.627503
997,0.035723,0.035723,0.035723,0.035723,0.035723,0.035723,0.035723,0.035723,0.035723,0.035723
998,1.515201,1.515201,1.515201,1.515201,1.515201,1.515201,1.515201,1.515201,1.515201,1.515201


In [38]:
df4 = pd.concat([df4, df4.T.stack().reset_index(name='new')['new']], axis=1)

In [39]:
df5 = pd.concat([df5, df5.T.stack().reset_index(name='new')['new']], axis=1)

In [40]:
data["beta"] = df4["new"]

In [41]:
data["gamma"] = df5["new"]

In [42]:
data

,suspectible,infected,recovered,beta,gamma
0,988.846594,6.045620e+00,5.107785,1.816185,0.908093
1,994.282358,3.350867e+00,2.366775,1.216265,0.608132
2,998.355467,1.321840e+00,0.322693,0.279756,0.139878
3,989.506310,5.719372e+00,4.774319,1.759483,0.879742
4,990.853755,5.052331e+00,4.093914,1.632993,0.816497
...,...,...,...,...,...
9995,202.845900,1.477279e-07,797.154100,0.914129,0.457065
9996,202.846227,4.801898e-04,797.153292,0.627503,0.313752
9997,990.194264,5.378921e+00,4.426814,0.035723,0.017861
9998,202.845900,-5.642916e-11,797.154100,1.515201,0.757600


In [43]:
data["recovered(O)"] = Df3["new"]

In [44]:
data["infected(O)"] = Df2["new"]

In [45]:
data["suspectible(O)"] = Df1["new"]

In [46]:
data

,suspectible,infected,recovered,beta,gamma,recovered(O),infected(O),suspectible(O)
0,988.846594,6.045620e+00,5.107785,1.816185,0.908093,13.904280,1.449376e+01,971.601965
1,994.282358,3.350867e+00,2.366775,1.216265,0.608132,5.157633,6.094350e+00,988.748017
2,998.355467,1.321840e+00,0.322693,0.279756,0.139878,0.521095,1.519511e+00,997.959394
3,989.506310,5.719372e+00,4.774319,1.759483,0.879742,12.719766,1.337379e+01,973.906445
4,990.853755,5.052331e+00,4.093914,1.632993,0.816497,10.398031,1.116270e+01,978.439265
...,...,...,...,...,...,...,...,...
9995,202.845900,1.477279e-07,797.154100,0.914129,0.457065,797.154100,1.130647e-07,202.845900
9996,202.846227,4.801898e-04,797.153292,0.627503,0.313752,797.153430,3.985035e-04,202.846172
9997,990.194264,5.378921e+00,4.426814,0.035723,0.017861,4.523736,5.473924e+00,990.002341
9998,202.845900,-5.642916e-11,797.154100,1.515201,0.757600,797.154100,-5.551872e-11,202.845900


In [64]:
X = data[["suspectible","infected","recovered","beta"]]
y = data[["recovered(O)","infected(O)","suspectible(O)"]]

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.optimizers import Adam
# load the dataset
# define the keras model
model = Sequential()
model.add(Dense(50, input_dim=4, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(20,  activation='relu'))
model.add(Dense(20,  activation='relu'))
model.add(Dense(10,  activation='relu'))
model.add(Dense(10,  activation='relu'))
model.add(Dense(10,  activation='relu'))
model.add(Dense(10,  activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='relu'))
# compile the keras model
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
8000/8000 [==============================] - 1s 154us/step - loss: 4115.9764 - accuracy: 0.9886
Epoch 2/20
8000/8000 [==============================] - 1s 147us/step - loss: 1366.5049 - accuracy: 0.9939
Epoch 3/20
8000/8000 [==============================] - 1s 146us/step - loss: 1418.9397 - accuracy: 0.9937
Epoch 4/20
8000/8000 [==============================] - 1s 148us/step - loss: 1372.8474 - accuracy: 0.9940
Epoch 5/20
8000/8000 [==============================] - 1s 146us/step - loss: 1344.0749 - accuracy: 0.9948
Epoch 6/20
8000/8000 [==============================] - 1s 151us/step - loss: 1166.3661 - accuracy: 0.9945
Epoch 7/20
8000/8000 [==============================] - 1s 148us/step - loss: 1185.0250 - accuracy: 0.9967
Epoch 8/20
8000/8000 [==============================] - 1s 146us/step - loss: 1119.8485 - accuracy: 0.9966
Epoch 9/20
8000/8000 [==============================] - 1s 149us/step - loss: 1028.1242 - accuracy: 0.9973
Epoch 10/20
8000/8000 [==============

In [68]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

2000/2000 [==============================] - 0s 30us/step
Accuracy: 99.95
